In [1]:
import numpy as np
import cv2
import time
import os
import yaml

import open3d as o3d

from wire_detector_platforms import WireDetectorCPU
import viz_utils as vu

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open('wire_detect_config.yaml', 'r') as file:
    detection_config = yaml.safe_load(file)

# Create a WireDetector instance
input_image_size = [480, 270]

# folder = "/media/tyler/Storage/field_tests/street_wire_experiments/wire_tracking_05-07_40fov/" 
folder = '/media/tyler/Storage/Research/Datasets/wire_tracking_05-07_40fov/'
# folder = "/home/tyler/Documents/wire_tracking_05-07_40fov/" 
# folder = "/media/tyler/Storage/field_tests/hawkins_2-19/wire_tracking_02-19_1/" 
# folder = "/media/tyler/Storage/field_tests/acfa_3-21/" 

rgb_folder = folder + 'rgb_images/'
depth_folder = folder + 'depth_images/'
camera_intrinsics_file = folder + 'rgb_camera_intrinsics.npy'
camera_intrinsics = np.load(camera_intrinsics_file)
rgb_file_list = sorted(os.listdir(rgb_folder))
depth_file_list = sorted(os.listdir(depth_folder))
original_image_size = cv2.imread(os.path.join(rgb_folder, rgb_file_list[0])).shape[:2][::-1]  # (width, height)

# resize the camera intrinsics to match the input image size
camera_intrinsics[0, 0] *= input_image_size[0] / original_image_size[0]
camera_intrinsics[1, 1] *= input_image_size[1] / original_image_size[1]
camera_intrinsics[0, 2] *= input_image_size[0] / original_image_size[0]
camera_intrinsics[1, 2] *= input_image_size[1] / original_image_size[1]

wire_detector = WireDetectorCPU(detection_config, camera_intrinsics)

wire_detection_dir = folder + 'ransac_results/'

if os.path.exists(wire_detection_dir) == False:
    os.makedirs(wire_detection_dir)

In [3]:
avg_frequency = 0.0
renderer, material = vu.create_renderer()

do_rendering = True

if do_rendering:
    sync_ransac_time = 0.0
    async_ransac_time = 0.0
    roi_time = 0.0
    detect_wire_time = 0.0
    for file in rgb_file_list:
        start_time = time.perf_counter()
        rgb_timestamp = int(file.split('.')[0])
        closest_depth_timestamp = None
        for depth_file in depth_file_list:
            if closest_depth_timestamp is None or abs(int(depth_file.split('.')[0]) - rgb_timestamp) < abs(int(closest_depth_timestamp.split('.')[0]) - rgb_timestamp):
                closest_depth_timestamp = depth_file
                closest_depth_file = os.path.join(depth_folder, str(closest_depth_timestamp))
                
        rgb_img = cv2.imread(os.path.join(rgb_folder, file))
        depth_img = np.load(closest_depth_file)
        rgb_img = cv2.resize(rgb_img, (input_image_size[0], input_image_size[1]))
        depth_img = cv2.resize(depth_img, (input_image_size[0], input_image_size[1]))
        
        min_depth = 0.5
        depth_img[depth_img <= min_depth] = 0

        start_time = time.perf_counter()
        start_detect_wire = time.perf_counter()
        wire_lines, wire_midpoints, avg_angle, midpoint_dists_wrt_center = wire_detector.detect_wires_2d(rgb_img)
        detect_wire_time += time.perf_counter() - start_detect_wire

        start_roi = time.perf_counter()
        regions_of_interest, roi_line_counts = wire_detector.find_regions_of_interest(depth_img, avg_angle, midpoint_dists_wrt_center)
        roi_time += time.perf_counter() - start_roi

        start_ransac_sync = time.perf_counter()
        fitted_lines, line_inlier_counts, roi_pcs, roi_point_colors, rgb_masked = wire_detector.ransac_on_rois(regions_of_interest, roi_line_counts, avg_angle, depth_img, viz_img=rgb_img)
        sync_ransac_time += time.perf_counter() - start_ransac_sync

        end_time = time.perf_counter()
        elapsed_time = end_time - start_time
        avg_frequency += 1.0 / elapsed_time

        rgb_masked = vu.draw_3d_line_on_image(rgb_masked, fitted_lines, camera_intrinsics, color=(0, 255, 0), thickness=3)

        if fitted_lines is None or len(fitted_lines) == 0 or roi_pcs is None or len(roi_pcs) == 0:
            print("No wires detected in image: ", rgb_timestamp)
            continue

        img_name_3d = os.path.join(wire_detection_dir, str(rgb_timestamp) + '_3d.png')
        vu.capture_fitted_lines_in_image(renderer, material, img_name_3d, fitted_lines, roi_pcs, roi_point_colors)
        img_name_2d = os.path.join(wire_detection_dir, str(rgb_timestamp) + '_2d.png')
        cv2.imwrite(img_name_2d, rgb_masked)

    avg_frequency /= len(rgb_file_list)
    avg_sync_ransac_time = sync_ransac_time / len(rgb_file_list)
    avg_roi_time = roi_time / len(rgb_file_list)
    avg_detect_wire_time = detect_wire_time / len(rgb_file_list)
    print(f"Detect 3D Average frequency: {avg_frequency:.4f}, Average Period: {1/avg_frequency:.4f} seconds")
    print(f"Average 2D Detect Wire frequency: {1/avg_detect_wire_time:.4f}, Average Period: {avg_detect_wire_time:.4f} seconds")
    print(f"Average ROI frequency: {1/avg_roi_time:.4f}, Average Period: {avg_roi_time:.4f} seconds")
    print(f"Average RANSAC frequency: {1/avg_sync_ransac_time:.4f}, Average Period: {avg_sync_ransac_time:.4f} seconds")


libEGL warning: egl: failed to create dri2 screen


[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x72d907443010 (threading is enabled)
EGL(1.5)
OpenGL(4.5)
Detect 3D Average frequency: 76.8298, Average Period: 0.0130 seconds
Average 2D Detect Wire frequency: 473.0969, Average Period: 0.0021 seconds
Average ROI frequency: 265.9128, Average Period: 0.0038 seconds
Average RANSAC frequency: 127.1075, Average Period: 0.0079 seconds


In [4]:
import shutil

# copy the ransac results into a seperate 2d and 3d folder
os.makedirs(folder + 'ransac_results_2d/', exist_ok=True)
video_2d_name = folder + 'ransac_results_2d.mp4'
os.makedirs(folder + 'ransac_results_3d/', exist_ok=True)
video_3d_name = folder + 'ransac_results_3d.mp4'

frames_2d = []
frames_3d = []

files = sorted(os.listdir(wire_detection_dir))

for file in files:
    if file.endswith('_2d.png'):
        shutil.copy(os.path.join(wire_detection_dir, file), os.path.join(folder + 'ransac_results_2d/', file))
        frames_2d.append(os.path.join(folder + 'ransac_results_2d/', file))

    elif file.endswith('_3d.png'):
        shutil.copy(os.path.join(wire_detection_dir, file), os.path.join(folder + 'ransac_results_3d/', file))
        frames_3d.append(os.path.join(folder + 'ransac_results_3d/', file))

vu.make_video(frames_2d, video_2d_name, fps=10)
vu.make_video(frames_3d, video_3d_name, fps=10)